In [2]:
import pandas as pd

# CONFIGURACIÓN
archivo_excel = '../data/raw/base_anuario.xlsx'
nombre_hoja = 'Base de datos'

# 1. CARGA Y LIMPIEZA
print(f"📂 Cargando archivo: {archivo_excel} [{nombre_hoja}]...")
try:
    df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)
except FileNotFoundError:
    print("❌ Error: No se encuentra el archivo.")
    exit()

# Aseguramos que las columnas numéricas sean números
cols_num = ['Matrícula Total', 'Egresados Total']
for col in cols_num:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# 2. MAPEO DE NIVELES
mapa_niveles = {
    'TÉCNICO SUPERIOR': 'Técnico Superior',
    'LICENCIATURA EN EDUCACIÓN NORMAL': 'Licenciatura',
    'LICENCIATURA UNIVERSITARIA Y TECNOLÓGICA': 'Licenciatura',
    'ESPECIALIDAD': 'Licenciatura',
    'MAESTRÍA': 'Maestría',
    'DOCTORADO': 'Doctorado'
}

df['NIVEL'] = df['NIVEL'].str.upper().str.strip()
df['Nivel_Agrupado'] = df['NIVEL'].map(mapa_niveles)

# --- CÁLCULOS BASE ---

# Paso A: Agrupar por Entidad, Nivel y Campo para obtener totales absolutos
df_campos = df.groupby(['ENTIDAD', 'Nivel_Agrupado', 'CAMPO AMPLIO'])[cols_num].sum().reset_index()

# Paso B: Calcular los totales por Nivel para sacar porcentajes
df_totales_nivel = df_campos.groupby(['ENTIDAD', 'Nivel_Agrupado'])[cols_num].sum().reset_index()
df_totales_nivel = df_totales_nivel.rename(columns={
    'Matrícula Total': 'Total_Mat_Nivel', 
    'Egresados Total': 'Total_Egr_Nivel'
})

# Unimos los totales al dataframe de campos
df_campos = df_campos.merge(df_totales_nivel, on=['ENTIDAD', 'Nivel_Agrupado'])

# Calculamos porcentajes de participación
df_campos['Part_Matricula'] = (df_campos['Matrícula Total'] / df_campos['Total_Mat_Nivel']) * 100
df_campos['Part_Egresados'] = (df_campos['Egresados Total'] / df_campos['Total_Egr_Nivel']) * 100


# --- GENERACIÓN DE LOS DOS TOP 3 ---

# 1. TOP 3 POR MATRÍCULA
df_top3_matricula = df_campos.sort_values(['ENTIDAD', 'Nivel_Agrupado', 'Matrícula Total'], ascending=[True, True, False])
df_top3_matricula = df_top3_matricula.groupby(['ENTIDAD', 'Nivel_Agrupado']).head(3)
# Seleccionamos solo columnas relevantes
df_top3_matricula = df_top3_matricula[['ENTIDAD', 'Nivel_Agrupado', 'CAMPO AMPLIO', 'Matrícula Total', 'Part_Matricula']]

# 2. TOP 3 POR EGRESADOS
df_top3_egresados = df_campos.sort_values(['ENTIDAD', 'Nivel_Agrupado', 'Egresados Total'], ascending=[True, True, False])
df_top3_egresados = df_top3_egresados.groupby(['ENTIDAD', 'Nivel_Agrupado']).head(3)
# Seleccionamos solo columnas relevantes
df_top3_egresados = df_top3_egresados[['ENTIDAD', 'Nivel_Agrupado', 'CAMPO AMPLIO', 'Egresados Total', 'Part_Egresados']]


# --- VISUALIZACIÓN DE RESULTADOS ---

print("\n--- RESUMEN 1: TOP 3 POR MATRÍCULA (Ejemplo: AGUASCALIENTES - Licenciatura) ---")
filtro_mat = (df_top3_matricula['ENTIDAD'] == 'AGUASCALIENTES') & (df_top3_matricula['Nivel_Agrupado'] == 'Licenciatura')
print(df_top3_matricula[filtro_mat])

print("\n--- RESUMEN 2: TOP 3 POR EGRESADOS (Ejemplo: AGUASCALIENTES - Licenciatura) ---")
filtro_egr = (df_top3_egresados['ENTIDAD'] == 'AGUASCALIENTES') & (df_top3_egresados['Nivel_Agrupado'] == 'Licenciatura')
print(df_top3_egresados[filtro_egr])

# Opcional: Exportar a Excel en hojas separadas
# with pd.ExcelWriter("Reporte_Educacion_Top3_Separado.xlsx") as writer:
#     df_top3_matricula.to_excel(writer, sheet_name="Top Matricula", index=False)
#     df_top3_egresados.to_excel(writer, sheet_name="Top Egresados", index=False)

📂 Cargando archivo: ../data/raw/base_anuario.xlsx [Base de datos]...

--- RESUMEN 1: TOP 3 POR MATRÍCULA (Ejemplo: AGUASCALIENTES - Licenciatura) ---
           ENTIDAD Nivel_Agrupado                            CAMPO AMPLIO  \
9   AGUASCALIENTES   Licenciatura               ADMINISTRACIÓN Y NEGOCIOS   
16  AGUASCALIENTES   Licenciatura  INGENIERÍA, MANUFACTURA Y CONSTRUCCIÓN   
12  AGUASCALIENTES   Licenciatura                    CIENCIAS DE LA SALUD   

    Matrícula Total  Part_Matricula  
9             13381       22.716624  
16            12647       21.470528  
12             8664       14.708679  

--- RESUMEN 2: TOP 3 POR EGRESADOS (Ejemplo: AGUASCALIENTES - Licenciatura) ---
           ENTIDAD Nivel_Agrupado                            CAMPO AMPLIO  \
9   AGUASCALIENTES   Licenciatura               ADMINISTRACIÓN Y NEGOCIOS   
16  AGUASCALIENTES   Licenciatura  INGENIERÍA, MANUFACTURA Y CONSTRUCCIÓN   
14  AGUASCALIENTES   Licenciatura             CIENCIAS SOCIALES Y DERECHO   
